# Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from src.dataset import PosterDataset
from src.train import train, test
from src.models import Simple3BlockCNN, BNDropout3BlockCNN
from src.utils import split_data, print_history

In [ ]:
BATCH_SIZE = 32
TRAIN_SIZE = 0.8
EPOCHS = 30
LEARNING_RATE = 1e-3
PATIENCE = 5

In [ ]:
train_loader, val_loader, test_loader = split_data("data", batch_size=BATCH_SIZE, train_size=TRAIN_SIZE)

# Training

In [ ]:
model = Simple3BlockCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2)

history = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=EPOCHS,
    patience=PATIENCE
)

In [ ]:
history_path = "models/run_3/history.json"
print_history(history_path)

# Testing

In [ ]:
simple_model = Simple3BlockCNN()
simple_model_path = "models/run_1/best_model.pth"
simple_model.load_state_dict(torch.load(simple_model_path))

enhanced_model = BNDropout3BlockCNN()
enhanced_model_path = "models/run_3/best_model.pth"
enhanced_model.load_state_dict(torch.load(enhanced_model_path))

criterion = nn.BCEWithLogitsLoss()

In [ ]:
simple_test = test(model=simple_model,
                   test_loader=test_loader,
                   criterion=criterion)

enhanced_test = test(model=enhanced_model,
                     test_loader=test_loader,
                     criterion=criterion)

In [ ]:
from tabulate import tabulate

table_data = [
    ["Simple3BlockCNN", round(simple_test[0], 3), simple_test[1]],
    ["BNDropout3BlockCNN", round(enhanced_test[0], 3), enhanced_test[1]],
]

headers = ["Model", "Loss", "Accuracy"]

print(tabulate(table_data, headers=headers))
